In [6]:
import os
import time
from riotwatcher import LolWatcher, ApiError
import pandas as pd

api_key = os.getenv('RIOT_API_KEY')
lol_watcher = LolWatcher(api_key)
my_region = 'NA1'

try:
    old_games = pd.read_csv("champ_role_data_both_s13.csv",index_col=0)
    old_games["start"] = pd.to_datetime(old_games["start"])
    old_max = old_games["start"].max()
    startTime = (old_max - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
except:
    startTime = 0

NA1 = "NA1"
ARAM = 450
RANKED_SOLO = 420

games_format = pd.DataFrame(columns=['account','match_id','start',
                        'ally_id_1','ally_id_2','ally_id_3','ally_id_4',
                        'enemy_id_1','enemy_id_2','enemy_id_3','enemy_id_4','enemy_id_5',
                        'my_champ','my_side','my_role','support_item',
                        'ally_champ_1','ally_champ_2','ally_champ_3','ally_champ_4',
                        'ally_role_1','ally_role_2','ally_role_3','ally_role_4',
                        'enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5',
                        'enemy_role_1','enemy_role_2','enemy_role_3','enemy_role_4','enemy_role_5',
                        'win'])

support_item_ids = [3862,3863,3864, ## ad damage
                    3850,3851,3852, ## ap damage
                    3858,3859,3860, ## ap tank
                    3854,3855,3856] ## ad tank

games = []

for summoner_name in ["duoking1","jahseh gus"]:
    me = lol_watcher.summoner.by_name(my_region,summoner_name)
    puuid = me['puuid']

    ## Try to get 500 ARAM match IDs of player
    matches = []
    for big_loop_i in range(2):
        for small_loop_i in range(5):
            start = 500*big_loop_i+100*small_loop_i
            matchlist = lol_watcher.match.matchlist_by_puuid(puuid=puuid,region=NA1,start_time=startTime,
                                                             start=start,queue=RANKED_SOLO,count=100)
            matches = matches + matchlist
        time.sleep(1.2)

    for match_id in matches:
        match = lol_watcher.match.by_id(region=NA1,match_id=match_id)
        participants = match["metadata"]["participants"]
        infodto = match["info"]["participants"]
        game_start = match["info"]["gameStartTimestamp"]
        ## Determine side of player (red or blue)
        if puuid in participants[:5]:
            blue = True
        else:
            blue = False

        ## Grabbing all of our teammates (not us!)
        ally_ids = ["","","",""]
        ally_champs = ["","","",""]
        ally_roles = ["","","",""]
        offset = 0 if blue else 5
        ii = 0
        for t in range(5):
            curr = participants[offset+t]
            if curr != puuid:
                ally_ids[ii] = curr
                ally_champs[ii] = infodto[offset+t]["championName"]
                ally_roles[ii] = infodto[offset+t]["teamPosition"]
                ii += 1
            else:
                ## write down our champ
                win = infodto[offset+t]["win"]
                my_champ = infodto[offset+t]["championName"]
                my_role = infodto[offset+t]["teamPosition"]
                support_item = False
                if my_champ == "Senna":
                    for item_index in range(6):
                        slot = "item" + str(item_index)
                        item = infodto[offset+t][slot]
                        if item in support_item_ids:
                            support_item = True
                            break

        ## For legibility, initializing variables with easy names (can rewrite for performance)
        account = summoner_name
        my_side = 'b' if blue else 'r'
        ally_id_1 = ally_ids[0] 
        ally_id_2 = ally_ids[1] 
        ally_id_3 = ally_ids[2] 
        ally_id_4 = ally_ids[3]
        enemy_id_1 = participants[5-offset]
        enemy_id_2 = participants[6-offset]
        enemy_id_3 = participants[7-offset]
        enemy_id_4 = participants[8-offset]
        enemy_id_5 = participants[9-offset]
        ally_champ_1 = ally_champs[0]
        ally_champ_2 = ally_champs[1]
        ally_champ_3 = ally_champs[2]
        ally_champ_4 = ally_champs[3]
        ally_role_1 = ally_roles[0]
        ally_role_2 = ally_roles[1]
        ally_role_3 = ally_roles[2]
        ally_role_4 = ally_roles[3]
        enemy_champ_1 = infodto[5-offset]["championName"]
        enemy_champ_2 = infodto[6-offset]["championName"]
        enemy_champ_3 = infodto[7-offset]["championName"]
        enemy_champ_4 = infodto[8-offset]["championName"]
        enemy_champ_5 = infodto[9-offset]["championName"]
        enemy_role_1 = infodto[5-offset]["teamPosition"]
        enemy_role_2 = infodto[6-offset]["teamPosition"]
        enemy_role_3 = infodto[7-offset]["teamPosition"]
        enemy_role_4 = infodto[8-offset]["teamPosition"]
        enemy_role_5 = infodto[9-offset]["teamPosition"]

        match_frame = pd.Series([account,match_id,game_start,
                                ally_id_1,ally_id_2,ally_id_3,ally_id_4,
                                enemy_id_1,enemy_id_2,enemy_id_3,enemy_id_4,enemy_id_5,
                                my_champ,my_side,my_role,support_item,
                                ally_champ_1,ally_champ_2,ally_champ_3,ally_champ_4,
                                ally_role_1,ally_role_2,ally_role_3,ally_role_4,
                                enemy_champ_1,enemy_champ_2,enemy_champ_3,enemy_champ_4,enemy_champ_5,
                                enemy_role_1,enemy_role_2,enemy_role_3,enemy_role_4,enemy_role_5,
                                win],
                                index=games_format.columns)
        games.append(match_frame)
        time.sleep(1.2)

## Convert list to pandas DF
games = pd.DataFrame(games)
games["start"] = pd.to_datetime(games['start'],unit='ms')

## Properly classify support
games.loc[games.support_item == True, 'my_role'] = "UTILITY"
games.loc[(games.support_item == True) & (games.ally_role_1 == "UTILITY"), "ally_role_1"] = "BOTTOM"
games.loc[(games.support_item == True) & (games.ally_role_2 == "UTILITY"), "ally_role_2"] = "BOTTOM"
games.loc[(games.support_item == True) & (games.ally_role_3 == "UTILITY"), "ally_role_3"] = "BOTTOM"
games.loc[(games.support_item == True) & (games.ally_role_4 == "UTILITY"), "ally_role_4"] = "BOTTOM"

In [7]:
games

,account,match_id,start,ally_id_1,ally_id_2,ally_id_3,ally_id_4,enemy_id_1,enemy_id_2,enemy_id_3,...,enemy_champ_2,enemy_champ_3,enemy_champ_4,enemy_champ_5,enemy_role_1,enemy_role_2,enemy_role_3,enemy_role_4,enemy_role_5,win
0,duoking1,NA1_4668697374,2023-05-29 01:35:16.352,5NSbn1N5I8YqQ-EHRfaIRlk8ldEVhh4nrdrRi45yvP2wuv...,lfvS16HOAw4ygi0QPSCP0cTbjR329U7-x_EoVwR67Kx9Oj...,DDMXDbfcJ88oETWS8eTNaCRKZVa_IJFipD6IY8o8hqT8QV...,oaK67A1gEj66GvU_rirkuJS7f0r0x-5B2x78Kkza9mWN6i...,Tqoz2qBxganJ42yqK7FBgiSAU2BluneDhxqBwoJEaK79tj...,1gDOJL8osdmvOT9gtnFF6h-Is0BxkDzQcXzLVjhe1aa-Yx...,t5nQZz3HNoGNVZmE0ILB0u5Fo-GnE6oD--_uScc918z4TG...,...,Hecarim,Lissandra,Ezreal,Karma,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False
1,duoking1,NA1_4668657521,2023-05-29 00:57:56.182,UL0CA30Nt9EuXQjKC-li93_m8EtqByI1JPVPb6fmESYQkV...,6x0rS2h-xajXqwUx-r9MDSd7sNsyRrHgLV778vlP3rLyzC...,t5nQZz3HNoGNVZmE0ILB0u5Fo-GnE6oD--_uScc918z4TG...,4-AaIsUMcm4F_Z64jUmAiEFnbWItAVFnva8xv-w6AIf5oP...,Tqoz2qBxganJ42yqK7FBgiSAU2BluneDhxqBwoJEaK79tj...,lfvS16HOAw4ygi0QPSCP0cTbjR329U7-x_EoVwR67Kx9Oj...,mhdXZ5CRlN1JzPWAlPV-1TFKbMHkuVKInFefhw79KZI9G6...,...,Neeko,Jayce,Jinx,Milio,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False
2,duoking1,NA1_4668622322,2023-05-29 00:16:56.853,NxYnAaiG_H4zrJT8rt6-9Ifkp5RgS-SpttQxTQ894bA0mk...,TP1lW1LJVg30U9NCzXoRR9VBOkMVzmadK3riBWZ-ByzkSi...,DWfO1MVZsxUL-LM37kL5eYtcQOdHhekQwkcM642ZuV2uuB...,jIIVdKK1KZ9sW-kbSyDIhpIs_pkUaVRYl_wh3JZHSqzq5-...,zZyEIee08LTy1RKK6hpBT1SFbAfOqYLuWMp34IA6eGIeMY...,UCXxCP2ua-dPopu9h3ez01hclCKj1uRUcsb2U5fQXjY6gy...,K2BBFCc773xCePD95_QgpumGz72_fcXy7f-So4MKB07oU3...,...,Udyr,Cassiopeia,Jhin,Rakan,TOP,JUNGLE,MIDDLE,BOTTOM,UTILITY,False


In [8]:
## Ally
games_ally = games[['account','start',
                    'my_champ','my_role',
                    'ally_champ_1','ally_champ_2','ally_champ_3','ally_champ_4',
                    'ally_role_1','ally_role_2','ally_role_3','ally_role_4',
                    'win']].copy()
games_ally["win"] = games_ally["win"].replace({True: 1, False: 0})
games_ally["count"] = 1
games_ally["ally"] = True
games_ally_new = []
for ii in range(4):
    ally_champ_i = 'ally_champ_' + str(ii+1)
    ally_role_i = 'ally_role_' + str(ii+1)
    new = games_ally[['account','start','my_champ','my_role','ally',ally_champ_i,ally_role_i,'win','count']]
    new.columns = ['account','start','my_champ','my_role','ally','champ','role','win','count']
    games_ally_new.append(new)
games_ally_new = pd.concat(games_ally_new,ignore_index=True)

In [9]:
## Enemy
games_enemy = games[['account','start',
                    'my_champ','my_role',
                    'enemy_champ_1','enemy_champ_2','enemy_champ_3','enemy_champ_4','enemy_champ_5',
                    'enemy_role_1','enemy_role_2','enemy_role_3','enemy_role_4','enemy_role_5',
                    'win']].copy()
games_enemy["win"] = games_enemy["win"].replace({True: 0, False: 1})
games_enemy["count"] = 1
games_enemy["ally"] = False
games_enemy_new = []
for ii in range(5):
    enemy_champ_i = 'enemy_champ_' + str(ii+1)
    enemy_role_i = 'enemy_role_' + str(ii+1)
    new = games_enemy[['account','start','my_champ','my_role','ally',enemy_champ_i,enemy_role_i,'win','count']]
    new.columns = ['account','start','my_champ','my_role','ally','champ','role','win','count']
    games_enemy_new.append(new)
games_enemy_new = pd.concat(games_enemy_new,ignore_index=True)

In [10]:
try:
    full_games = pd.concat([games_ally_new,games_enemy_new,old_games])
except:
    full_games = pd.concat([games_ally_new,games_enemy_new])

In [11]:
full_games = full_games.reset_index(drop=True)
full_games.to_csv('champ_role_data_both_s13.csv')